In [1]:
import numpy as np
from dmipy.core.modeling_framework import MultiCompartmentModel
from dmipy.signal_models import sphere_models, cylinder_models, gaussian_models

from microtool.dmipy import DmipyTissueModel
from microtool.optimize import optimize_scheme
from microtool.utils import IO

## Setting up the VERDICT model

In [2]:
sphere = sphere_models.S4SphereGaussianPhaseApproximation(diffusion_constant=2.0e-9, diameter=10e-6)
ball = gaussian_models.G1Ball(lambda_iso=2.0e-9)
stick = cylinder_models.C1Stick(mu=[np.pi / 2, np.pi / 2], lambda_par=8.0e-9)

verdict_model = MultiCompartmentModel(models=[sphere, ball, stick])
# In Bonet-Carne et. al. the diameter of the intracellular sphere (d_IC) and the vascular diffusivity (P)
# can be fixed such that we can estimate d_{EES}
verdict_model.set_fixed_parameter('S4SphereGaussianPhaseApproximation_1_diameter', 2.0e-9)
verdict_model.set_fixed_parameter('C1Stick_1_lambda_par', 8.0e-9)

verdict_model.set_fixed_parameter('C1Stick_1_mu', [np.pi / 2, np.pi / 2])

verdict_model = DmipyTissueModel(verdict_model, [.3, .3, .4])
print(verdict_model)

Tissue model with 9 scalar parameters:
Tissue-parameter                                 Value    Scale  Optimize    Fit    Fit Bounds
---------------------------------------------  -------  -------  ----------  -----  ------------------------------------------------
S4SphereGaussianPhaseApproximation_1_diameter   1e-05     1e-06  False       True   (1e-08, 1.9999999999999998e-05)
G1Ball_1_lambda_iso                             2e-09     1e-09  True        True   (1.0000000000000002e-10, 3.0000000000000004e-09)
C1Stick_1_mu_0                                  1.5708    1      False       True   (0.0, 3.141592653589793)
C1Stick_1_mu_1                                  1.5708    1      False       True   (-3.141592653589793, 3.141592653589793)
C1Stick_1_lambda_par                            8e-09     1e-09  False       True   (1.0000000000000002e-10, 3.0000000000000004e-09)
partial_volume_0                                0.3       1      True        True   (0.0, 1.0)
partial_volume_1       

## Setting up the acquisition scheme

In [3]:
from microtool.utils.saved_schemes import alexander_optimal_perturbed

scheme = alexander_optimal_perturbed()
print(scheme)

Acquisition scheme with 120 measurements and 6 scalar parameters. 
total number of optimized parameters is 12:
  DiffusionPulseMagnitude [mT/mm] in (0.0, 5000.0)    DiffusionGradientAnglePhi [rad] (fixed)    DiffusionGradientAngleTheta [rad] (fixed)    DiffusionPulseWidth [s] in (0.001, 0.1)    DiffusionPulseInterval [s] in (0.001, 0.1)    EchoTime [s] (fixed)
--------------------------------------------------  -----------------------------------------  -------------------------------------------  -----------------------------------------  --------------------------------------------  ----------------------
                                             0.19                                  -0.262465                                      1.5708                                    0.019                                             0.026                     0.1
                                             0.19                                   0.610784                                      1.0

## Optimization

In [5]:
scheme, _ = optimize_scheme(scheme, verdict_model, .02, method="trust-constr", solver_options={"verbose":2,"maxiter":100})
print(scheme)
IO.save_pickle(scheme, 'schemes/optimized_scheme.pkl')

| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  |
|-------|-------|-------|-------------|----------|----------|----------|
|   1   |  13   |   0   | +1.0592e+06 | 1.00e+00 | 4.03e+06 | 0.00e+00 |


C:\Users\Frank\miniconda3\envs\devmicro\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


|   2   |  26   |   1   | +1.7935e+02 | 1.00e+00 | 6.31e+02 | 0.00e+00 |
|   3   |  39   |  11   | +1.7930e+02 | 1.00e+00 | 6.31e+02 | 0.00e+00 |
|   4   |  52   |  23   | +1.4281e+02 | 1.78e+00 | 3.56e+02 | 0.00e+00 |
|   5   |  65   |  35   | +1.3242e+02 | 1.78e+00 | 2.78e+02 | 0.00e+00 |
|   6   |  78   |  47   | +1.2667e+02 | 1.78e+00 | 2.24e+02 | 0.00e+00 |
|   7   |  91   |  59   | +1.2547e+02 | 1.78e+00 | 2.03e+02 | 0.00e+00 |
|   8   |  104  |  71   | +1.2535e+02 | 1.78e+00 | 1.97e+02 | 0.00e+00 |
|   9   |  117  |  83   | +1.2533e+02 | 1.78e+00 | 1.97e+02 | 0.00e+00 |
|  10   |  130  |  95   | +1.2487e+02 | 1.78e+00 | 1.97e+02 | 0.00e+00 |
|  11   |  143  |  107  | +1.0060e+02 | 1.78e+00 | 2.41e+02 | 0.00e+00 |
|  12   |  156  |  119  | +5.6439e+01 | 3.78e+00 | 1.24e+02 | 0.00e+00 |
|  13   |  169  |  131  | +3.9899e+01 | 3.78e+00 | 6.64e+01 | 0.00e+00 |
|  14   |  182  |  143  | +3.5562e+01 | 3.78e+00 | 6.85e+01 | 0.00e+00 |
|  15   |  195  |  155  | +2.0157e+01 | 3.78e+00 | 

C:\Users\Frank\OneDrive - Universiteit Utrecht\Documents\Major Research\Software\microtool\microtool\optimize\optimize.py:189: UserWarning: Optimization procedure was unsuccessful. Possible solutions include but are not limited to: Changing the optimizer setings, changing the optimization method or changing the initial scheme to a more suitable one.If you are using a scipy optimizer its settings can be changed by passing options to this function. If you are using a microtool optimization method please consult the optimization_methods module for more details.
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: 'schemes/optimized_scheme.pkl'